In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow import keras 
from keras import Sequential
from keras.layers import Dense
import joblib
from sklearn.metrics import confusion_matrix, f1_score, classification_report, balanced_accuracy_score, cohen_kappa_score

In [2]:
def saveModel(model, filename):
    model.save(filename)

def loadModel(filename):
    model = joblib.load(filename)
    return model

In [3]:
df = pd.read_csv("../[CLEANED]kepler-data.csv")
df.drop(columns = ["Unnamed: 0"], inplace=True)

In [4]:
ALL_COLUMNS = df.columns
ERROR_COLUMNS = [col for col in ALL_COLUMNS if "err" in col]
EXCLUDE = ["rowid", "kepid", "kepoi_name", "koi_score", "koi_disposition", "koi_pdisposition", "koi_tce_delivname", "koi_tce_plnt_num"] #+ ERROR_COLUMNS
TO_USE = list(set(ALL_COLUMNS) - set(EXCLUDE))

In [5]:
subset_df = df[df["koi_disposition"] != "CANDIDATE"]

In [6]:
X = subset_df[TO_USE].values
y = subset_df["koi_disposition"].apply(lambda x: x=='CONFIRMED').astype(int).values

In [7]:
X = StandardScaler().fit_transform(X)
X = PCA(n_components=30).fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0, stratify=y)

In [9]:
model = Sequential([
                    Dense(128, activation = 'relu'),
                    Dense(256, activation = 'relu'),
                    Dense(128, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
                   ])

In [10]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
146/146 [==============================] - 0s 2ms/step - loss: 0.1290 - accuracy: 0.9548
Epoch 2/20
146/146 [==============================] - 0s 2ms/step - loss: 0.0499 - accuracy: 0.9869
Epoch 3/20
146/146 [==============================] - 0s 2ms/step - loss: 0.0389 - accuracy: 0.9892
Epoch 4/20
146/146 [==============================] - 0s 2ms/step - loss: 0.0289 - accuracy: 0.9908
Epoch 5/20
146/146 [==============================] - 0s 2ms/step - loss: 0.0304 - accuracy: 0.9916
Epoch 6/20
146/146 [==============================] - 0s 2ms/step - loss: 0.0359 - accuracy: 0.9892
Epoch 7/20
146/146 [==============================] - 0s 3ms/step - loss: 0.0237 - accuracy: 0.9923
Epoch 8/20
146/146 [==============================] - 0s 3ms/step - loss: 0.0166 - accuracy: 0.9953
Epoch 9/20
146/146 [==============================] - 0s 3ms/step - loss: 0.0126 - accuracy: 0.9953
Epoch 10/20
146/146 [==============================] - 0s 2ms/step - loss: 0.0136 - accuracy: 0.9957

In [12]:
pred = model.predict(X_test)
pred = pred >= 0.5

In [13]:
def performance(test, pred):
    conf_matrix = confusion_matrix(test, pred)
    f1 = f1_score(test, pred)
    report = classification_report(test, pred)
    accuracy = balanced_accuracy_score(test, pred)
    kappa = cohen_kappa_score(test, pred)
    print(f"F1 Score: {f1}")
    print(f"Kappa Score: {kappa}")
    print(f"Accuracy Score: {accuracy}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(report)

In [14]:
performance(y_test, pred)

F1 Score: 0.9873417721518987
Kappa Score: 0.9811714304260337
Accuracy Score: 0.9887755841145671
Confusion Matrix:
[[1530    4]
 [  15  741]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1534
           1       0.99      0.98      0.99       756

    accuracy                           0.99      2290
   macro avg       0.99      0.99      0.99      2290
weighted avg       0.99      0.99      0.99      2290



In [15]:
saveModel(model, "../model/nn-model-error.h5")